# A simple Backtracking Constraint Solver

The module `ast` (short for <em><u>a</u>bstract <u>s</u>yntax <u>t</u>ree</em>) implements the method `parse`.  This method can be used to parse arbitrary Python expressions.  This method returns the syntax tree of the given expression.  Via the method `ast.walk` we can visit all nodes of this tree.

In [1]:
import ast

The function `collect_variables(expr)` takes a string `expr` that can be interpreted as a Python expression as input and collects all variables occurring in `expr`.  It takes care to remove the function symbols from the names returned by `collect_variables`.

In [2]:
def collect_variables(expr): 
    tree = ast.parse(expr)
    return { node.id for node in ast.walk(tree) 
                     if  isinstance(node, ast.Name) 
                     if  node.id not in dir(__builtins__)
           }

Below, the predefined function `dir` returns a list containing all predefined variables, functions, and classes. 

In [3]:
dir(__builtins__)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BaseExceptionGroup',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EncodingWarning',
 'EnvironmentError',
 'Exception',
 'ExceptionGroup',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'Runti

In [4]:
collect_variables('abs(x - y) + abs(z1 - z2)')

{'x', 'y', 'z1', 'z2'}

The input to the function `solve` is a *constraint satisfaction problem*, i.e. a *CSP*.  
The function `solve` tries to compute a solution of this problem via *backtracking*.
Its main purpose is to transform the given *CSP* into an *annotated CSP* where all the formulas 
are *annotated* with their variables, i.e. the third component of the *CSP* is now no longer a set of formulas but rather a set of pairs of the form `(f, V)` where `f` is a formula and `V` is the set of variables occurring in this formula.  The function then calls the auxilliary function `backtrack_search` that recursively solves the *annotated CSP*.

In [5]:
def solve(CSP):
    'Compute a solution for the given constraint satisfaction problem.'
    Variables, Values, Constraints = CSP
    CSP = (Variables,
           Values,
           [(f, collect_variables(f) & set(Variables)) for f in Constraints]
          )
    return backtrack_search({}, CSP)

Given a **consistent** *partial variable assignment* `Assignment` and a constraint satisfaction problem `CSP`,
the function `backtrack_search(Assignment, CSP)` tries to extend the given assignment recursively and thereby produce a solution of the given CSP.

In [ ]:
def backtrack_search(Assignment, CSP):
    '''
    Given a partial variable assignment, this function tries to 
    complete this assignment towards a solution of the CSP.
    '''
    Variables, Values, Constraints = CSP
    # If all variables are alrerady assigned, then Assignment is a solution
    # because it is partially consistent.
    if len(Assignment) == len(Variables): 
        return Assignment
    # take first unassigned variable
    var = [x for x in Variables if x not in Assignment][0]
    # try all values for this variable
    for value in Values:
        # If the assignment var ↦ value is consistent,
        # extend Assignment such that Assignment[var] = value.
        if isConsistent(var, value, Assignment, Constraints):
            NewAssign      = Assignment.copy()
            NewAssign[var] = value
            Solution = backtrack_search(NewAssign, CSP)
            if Solution != None:
                return Solution
            # else try next value
    # all values tested, but no solution found
    return None 

The function `isConsistent` takes four arguments:
- `var` is a variable.
- `value` is a value that is to be assigned to the variable `var`.
- `Assignment` is a partial variable assignment that does not assign a value for `var`
  and that is *consistent* with all constraints in the set `Constraints`.
- `Constraints` is a set of logical formulas.

The function checks whether the assignment 
$$ \texttt{Assignment} \cup \{\texttt{var} \mapsto \texttt{value}\}$$
violates any of the formulas in `Constraints`.  It assumes that
`Assignment` is *consistent*.

In [ ]:
def isConsistent(var, value, Assignment, Constraints):
    NewA      = Assignment.copy()
    NewA[var] = value
    return all(eval(f, globals(), NewA) for (f, Vs) in Constraints
                                        if var in Vs and Vs <= NewA.keys()
              )

The predefined function `eval` takes a Python expression and a 
variable assignment and evaluates the given expression.

In [6]:
eval('x + y == 7 * z', globals(), { 'x': 3, 'y': 4, 'z': 1 })

True

The function `globals` returns a dictionary containing all the variables and functions that are defined globally.

In [7]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import ast',
  'def collect_variables(expr): \n    tree = ast.parse(expr)\n    return { node.id for node in ast.walk(tree) \n                     if  isinstance(node, ast.Name) \n                     if  node.id not in dir(__builtins__)\n           }',
  'dir(__builtins__)',
  "collect_variables('abs(x - y) + abs(z1 - z2)')",
  "def solve(CSP):\n    'Compute a solution for the given constraint satisfaction problem.'\n    Variables, Values, Constraints = CSP\n    CSP = (Variables,\n           Values,\n           [(f, collect_variables(f) & set(Variables)) for f in Constraints]\n          )\n    return backtrack_search({}, CSP)",
  "eval('x + y == 7 * z', globals(), { 'x': 3, 'y': 4, 'z': 1 })",
  'globals()'],
 